## 1) Creación de dataset entrenamiento

### Librerias

In [ ]:
#importo librerias
import glob
import os
from tqdm import tqdm


### Instalacion de pysatproc (correr solo una vez)

In [ ]:
#!pip instal pysatproc

### Variables para configuracion de satproc_extract_chips

In [ ]:

dates="2021-01-01_2021-03-01"                                      # nombre de la carpeta con las imagenes
zona = "zona_3"                                                    # zona con la que vamos a trabajar
version =  2                                                       # version de la corrida actual 
size = 100                                                         # tamaño (pixeles) de las imagenes que se van a generar
step_size = 50                                                     # tamaño de la ventana
vector_file_path = "../data/shp/gt/R3/Asentamiento_R3_v2.geojson"  # ruta y archivo con la verdad de campo

###### estas variables no modifican #####
path_to_files = os.path.join("../images-S2",zona,dates,"*.tif")    # ruta a la carpeta con las imagenes satelitales
vector_aoi_file_path = vector_file_path
output_folder = os.path.join("../dataset/data_train",zona,"v"+str(version),str(size)+"_"+str(step_size))         # Destination path <ndim>/<size>_<step-size>/


### Lista de imagenes que van a ser procesadas

In [ ]:
!ls $path_to_files

### Ayuda de satproc_extract_chips

In [ ]:
#satproc_extract_chips --help

### Generacion del dataset para entrenamiento

El modelo de ML utiliza para entrenar un dataset compuesto por imágenes y máscaras binarias que delimitan el objeto de interés. Estas se generan vía la herramienta **satproc_extract_chips**. Para el caso del dataset de entrenamiento se generan imágenes y máscaras (utilizando las anotaciones de verdad de campo) y para el de predicción solo imágenes.

Para generar las imágenes de entrenamiento:

In [ ]:
#Creo el dataset para entrenamiento (imag + anotaciones)
!satproc_extract_chips \
    $path_to_files \
    -o  $output_folder \
    --size $size \
    --step-size $step_size \
    --aoi $vector_file_path \
    --labels $vector_file_path \
    --label-property 'class' \ # nombre del campo que tiene la clase a predecir
    --classes 'A' \            # valor que toma la clase a predecir
    --rescale \
    --rescale-mode s2_rgb_extra --lower-cut 1 --upper-cut 100


###### Los argumentos:

* **El primer argumento** es la ruta a las imágenes 

* **o** es la ruta de destino 

Recomendamos que dicha ruta sea descriptiva, por ejemplo “data_train/600_600/ ” describe : Data_train → datos usados para entrenar; 400_400 → <tamaño de la imagen >_ <tamaño del step-size> (las imágenes son cuadradas)

* **size** tamaño de las imágenes resultantes (las imágenes son cuadradas) 
* **step-size** paso del proceso. Si *step-size* es igual que el *size* entonces no hay overlap en las imágenes. 

En ocaciones es útil para el entrenamiento generar los chips con un overlap de este modo tenemos más datos para entrenar. Pero en la predicción valor debe ser igual al tamaño que la imagen 

* **crs** epsg: le asigna un epsg a la imagen 

* **label-property** nombre del campo donde se define cada categoría (solo se usa para el entrenamiento) 

* **classes** nombres de las clases (como aparecen en el geojson), separados por espacios

* **aoi** ruta al archivo vectorial donde están definidas las localidades. Al definir una region de interés solo se procesan las imágenes que interceptan esas localidades.

* **rescale** lleva los valores de las bandas a 0-255

Este comando va a generar dos carpetas en la ruta de destino : “images” y “masks”. Los archivos de la primera van a ser de tipo Tiff de 3 bandas (rgb) y los de la segunda van a ser, también, de tipo Tiff pero de N bandas donde N representa el número de clases, en este caso sólo una. Y donde cada una de las bandas es una máscara binaria


# 2) Entrenamiento

Generamos el entrenamiento del modelo utilizando los datasets creados en el paso previo. 
El modelo es una red neuronal CNN basado en la arquitectura **U-Net**. Este considera las imágenes y las máscaras binarias como inputs y genera una imagen con la probabilidad de encontrar al objeto de interés.   


### Instalacion de unet (correr solo una vez)

In [ ]:
#!pip install unetseg

Las librerías que importamos nos permite acceder a las herramientas que vamos a utilizar a lo largo del proyecto como por ejemplo la función de train y predict  del modelo de Unet.

### Librerias

In [ ]:
#importo la red neuronal
from unetseg.train import TrainConfig, train
from unetseg.evaluate import plot_data_generator
import os

### Variables para configuracion de unet

En esta etapa debemos definir la configuración del modelo de ML

In [ ]:
spe=100          # step per epoch (debe ser multipo de 16 y estar entre 80 y 320)
size_unet=160    # tamaño en la red (img/batch_size 10%)

config = TrainConfig(width=size_unet,  
                     height=size_unet, 
                     n_channels=4, 
                     n_classes=1,
                     apply_image_augmentation=True,
                     seed=42,
                     epochs=30, 
                     batch_size=16, 
                     steps_per_epoch=spe, 
                     early_stopping_patience=10,
                     validation_split=0.1, 
                     test_split=0.1,  
                     model_architecture='unet', #unetplusplus
                     images_path=os.path.join("../dataset/data_train",zona,'v'+str(version),str(size)+"_"+str(step_size)),                     
                     model_path=os.path.join('../data/weights/model',zona+'_UNet_techo_4D_spe'+str(spe)+'_img'+str(size)+'_size'+str(size)+'_sz'+str(step_size)+'.h5'),
                     evaluate=True ,
                     class_weights = [1]) 

*Obs:* Es util usar un nombre para el archivo de pesos que de informacion sobre los parametros de entrenamiento. por ejemplo: < modelo >_< proyecto >_< dim_de_las_imagenes >_< size >_< step_size >_< step_per_epoch >.h5 o similares 

### Grafico de 3 tiles al azar
Podemos visualizar alguna de las imágenes y máscaras.

In [ ]:
plot_data_generator(num_samples=3, fig_size=(10, 10), train_config = config,img_ch = 3)

### Corro el entrenamiento

In [ ]:
#Corro el modelo con los parametros establecidos
res_config = train(config)

### Graficos Loss y Mean_iou de train/test

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,4))

plt.subplot(121)
plt.plot(res_config.history['loss'])
plt.plot(res_config.history['val_loss'])
plt.title('Loss')

plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')

plt.subplot(122)
plt.plot(res_config.history['mean_io_u'])
plt.plot(res_config.history['val_mean_io_u'])
plt.title('mean_iou')
plt.ylabel('val_mean_iou')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')


plt.show()